In [1]:
# Map explorer for a single day using the tuned RandomForest
# (adapted to CSV with 'time', 'TX', 'era5_temp_max', 'delta_temp', ...)

import os
import numpy as np
import pandas as pd
import joblib
import folium
from matplotlib import cm, colors

# ---------------------------------------------------------------------
# 1. User parameters
# ---------------------------------------------------------------------
DAY = "2023-06-15"           # <-- set the day you want to explore (YYYY-MM-DD)

DATA_PATH  = "euro_final_dataset_v2_filtered.csv"   # adjust if needed
OUTPUT_DIR = "training_results"
MODEL_PATH = os.path.join(OUTPUT_DIR, "best_random_forest_tuned.joblib")

# ---------------------------------------------------------------------
# 2. Load data and model, recreate features
# ---------------------------------------------------------------------
df = pd.read_csv(DATA_PATH)

# Parse the 'time' column as datetime
if "time" not in df.columns:
    raise ValueError("Expected a 'time' column in the CSV, but it was not found.")

df["time"] = pd.to_datetime(df["time"])
all_station_ids = set(df["STAID"].unique())
n_all_stations = len(all_station_ids)

day_mask = df["time"].dt.date == pd.to_datetime(DAY).date()
df_day = df[day_mask].copy()

# Diagnostics: how many stations on this day vs total
day_station_ids = set(df_day["STAID"].unique())
n_day_stations = len(day_station_ids)
missing_station_ids = all_station_ids - day_station_ids
n_missing_stations = len(missing_station_ids)

print(f"Total unique stations in dataset: {n_all_stations}")
print(f"Stations with at least one record on {DAY}: {n_day_stations}")
print(f"Stations with NO record on {DAY}: {n_missing_stations}")

# Optional: show first few missing STAID
missing_preview = list(missing_station_ids)[:10]
print("Example missing station IDs on that day:", missing_preview)

if len(df_day) == 0:
    raise ValueError(f"No rows found for day {DAY} in the dataset.")

print(f"Rows for {DAY}: {len(df_day):,}")

# Recreate log features exactly as in training
for col in ["era5_precip", "rain_7day_avg", "wind_speed"]:
    df_day[col] = df_day[col].clip(lower=0)

df_day["era5_precip_log1p"]   = np.log1p(df_day["era5_precip"])
df_day["rain_7day_avg_log1p"] = np.log1p(df_day["rain_7day_avg"])
df_day["wind_speed_log1p"]    = np.log1p(df_day["wind_speed"])

FEATURES = [
    "latitude", "longitude", "elevation",
    "ndvi_local", "ndvi_global",
    "perc_urban", "perc_suburban", "perc_forest",
    "era5_u10", "era5_v10", "wind_speed", "wind_speed_log1p",
    "era5_precip", "era5_precip_log1p",
    "rain_7day_avg", "rain_7day_avg_log1p",
    "sin_day", "cos_day",
]
TARGET = "delta_temp"  # station TX - ERA5 Tmax

X_day = df_day[FEATURES].values

rf = joblib.load(MODEL_PATH)
print("Loaded model:", rf)

# Predict delta and corrected Tmax for this day
df_day["delta_obs"]  = df_day[TARGET]
df_day["delta_pred"] = rf.predict(X_day)
df_day["TX_corr"]    = df_day["era5_temp_max"] + df_day["delta_pred"]

# ---------------------------------------------------------------------
# 3. Build an interactive Folium map for this day
# ---------------------------------------------------------------------
center_lat = df_day["latitude"].mean()
center_lon = df_day["longitude"].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=4, tiles="CartoDB positron")

# Color scale for predicted delta (station - ERA5)
vmin = df_day["delta_pred"].min()
vmax = df_day["delta_pred"].max()
norm = colors.Normalize(vmin=vmin, vmax=vmax)
cmap = cm.get_cmap("coolwarm")

def color_for_value(v):
    rgba = cmap(norm(v))
    return colors.to_hex(rgba)

for _, row in df_day.iterrows():
    lat = row["latitude"]
    lon = row["longitude"]
    d_obs = row["delta_obs"]
    d_pred = row["delta_pred"]
    ndvi = row["ndvi_local"]
    urban = row["perc_urban"]
    elev = row["elevation"]
    era5 = row["era5_temp_max"]
    tx   = row["TX"]
    tx_corr = row["TX_corr"]

    popup_html = (
        f"<b>Station ID:</b> {row.get('STAID', 'NA')}<br>"
        f"<b>Date:</b> {DAY}<br>"
        f"<b>Station Tmax (TX):</b> {tx:.2f} °C<br>"
        f"<b>ERA5 Tmax:</b> {era5:.2f} °C<br>"
        f"<b>Observed delta (station−ERA5):</b> {d_obs:.2f} °C<br>"
        f"<b>Predicted delta (RF):</b> {d_pred:.2f} °C<br>"
        f"<b>Corrected Tmax:</b> {tx_corr:.2f} °C<br>"
        f"<b>Elevation:</b> {elev:.0f} m<br>"
        f"<b>Local NDVI:</b> {ndvi:.2f}<br>"
        f"<b>Urban fraction:</b> {urban:.2f}"
    )

    folium.CircleMarker(
        location=[lat, lon],
        radius=4,
        color=None,
        fill=True,
        fill_color=color_for_value(d_pred),
        fill_opacity=0.8,
        popup=folium.Popup(popup_html, max_width=300),
        tooltip=f"Δ_pred={d_pred:.2f} °C",
    ).add_to(m)

# Static legend
legend_html = f"""
<div style="
    position: fixed;
    bottom: 30px; left: 30px; width: 220px; height: 90px;
    background-color: white; border:2px solid grey; z-index:9999;
    font-size:12px; padding: 5px;
">
<b>Predicted bias Δ (station−ERA5)</b><br>
min: {vmin:.2f} °C<br>
max: {vmax:.2f} °C<br>
(colour shows Δ_pred at each station)
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

# Save to HTML
os.makedirs(OUTPUT_DIR, exist_ok=True)
html_path = os.path.join(OUTPUT_DIR, f"delta_map_{DAY}.html")
m.save(html_path)
print("Saved interactive map to:", html_path)


Total unique stations in dataset: 1891
Stations with at least one record on 2023-06-15: 1515
Stations with NO record on 2023-06-15: 376
Example missing station IDs on that day: [26676, 26682, 16459, 16463, 16469, 16476, 16479, 16487, 16489, 16494]
Rows for 2023-06-15: 1,515
Loaded model: RandomForestRegressor(max_depth=30, max_features=0.5, min_samples_leaf=2,
                      n_estimators=200, n_jobs=-1, random_state=42)


c:\Users\drugm\anaconda3\envs\genhack\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
C:\Users\drugm\AppData\Local\Temp\ipykernel_62572\2114737242.py:95: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap("coolwarm")


Saved interactive map to: training_results\delta_map_2023-06-15.html
